* digit28 only 2 features (gkernel BN) 5-s100zz
* digit28 only 2 features (gkernel BN) 5-s101zz
* digit28 only 2 features (gkernel BN) 5-s102zz

In [ ]:
ls -la ../input

In [ ]:
import datetime
now = datetime.datetime.now()
print(now)

In [ ]:
%matplotlib inline
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

In [ ]:
import datetime
import os.path
import itertools
from itertools import chain

import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn import cluster, datasets, mixture
from sklearn.datasets import load_digits
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns

import tensorflow as tf

from keras.layers import Input, Embedding, LSTM, GRU, Dense, Dropout, Lambda, \
    Conv1D, Conv2D, Conv3D, \
    Conv2DTranspose, \
    AveragePooling1D, AveragePooling2D, \
    MaxPooling1D, MaxPooling2D, MaxPooling3D, \
    GlobalAveragePooling1D, GlobalAveragePooling2D, \
    GlobalMaxPooling1D, GlobalMaxPooling2D, GlobalMaxPooling3D, \
    LocallyConnected1D, LocallyConnected2D, \
    concatenate, Flatten, Average, Activation, \
    RepeatVector, Permute, Reshape, Dot, \
    multiply, dot, add, \
    PReLU, \
    Bidirectional, TimeDistributed, \
    SpatialDropout1D, \
    BatchNormalization
from keras.models import Model, Sequential
from keras import losses
from keras.callbacks import BaseLogger, ProgbarLogger, Callback, History
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from keras.wrappers.scikit_learn import KerasClassifier
from keras import regularizers
from keras import initializers
from keras.metrics import categorical_accuracy
from keras.constraints import maxnorm, non_neg
from keras.optimizers import RMSprop
from keras.utils import to_categorical, plot_model
from keras import backend as K
import keras

In [ ]:
from PIL import Image
from zipfile import ZipFile
import h5py
import cv2
from tqdm import tqdm
import datetime

In [ ]:
# Load the data
train = pd.read_csv("../input/digit-recognizer/train.csv")
test = pd.read_csv("../input/digit-recognizer/test.csv")

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
y_train = train['label'].values
print(y_train.shape)

y_cat = to_categorical(y_train)
y_cat.shape

In [ ]:
x_train = train.iloc[:,1:].values / 255.0
print((x_train.min(), x_train.max()))
x_train.shape

In [ ]:
x_train_fl = x_train.reshape((x_train.shape[0], -1))
x_train_fl.shape

In [ ]:
plt.imshow(x_train[0].reshape((28,28)))

In [ ]:
x_test = test.iloc[:,:].values / 255.0
print((x_test.min(), x_test.max()))
x_test.shape

In [ ]:
x_test_fl = x_test.reshape((x_test.shape[0], -1))
x_test_fl.shape

In [ ]:
plt.imshow(x_test[0].reshape((28,28)))

In [ ]:
sample_submit = pd.read_csv("../input/digit-recognizer/sample_submission.csv")
print(sample_submit.shape)
sample_submit.head()

In [ ]:
def get_pred(src, rs=10001, show=False):
    print(src, rs)
    y_pred0 = pd.read_csv(os.path.join(src, 'proba_s{}.csv'.format(rs)))
    if show:
        print(y_pred0.shape)
        print(y_pred0.head())
    y_pred0_test = pd.read_csv(os.path.join(src, 'proba_test_s{}.csv'.format(rs)))
    if show:
        print(y_pred0_test.shape)
        print(y_pred0_test.head())
    
    return y_pred0, y_pred0_test

## -###-

In [ ]:
y_pred_0, y_pred_test_0 = get_pred('../input/digit28-only-2-features-gkernel-bn-5-s100zz', rs=10001, show=True)

In [ ]:
y_pred_list = []
y_pred_test_list = []

src = '../input/digit28-only-2-features-gkernel-bn-5-s100zz'
rs_init = 10001
for ii in range(30):
    rs = rs_init + ii
    y_pred_0, y_pred_test_0 = get_pred(src, rs)
    y_pred_list.append(y_pred_0)
    y_pred_test_list.append(y_pred_test_0)

src = '../input/digit28-only-2-features-gkernel-bn-5-s101zz'
rs_init = 10101
for ii in range(30):
    rs = rs_init + ii
    y_pred_0, y_pred_test_0 = get_pred(src, rs)
    y_pred_list.append(y_pred_0)
    y_pred_test_list.append(y_pred_test_0)

src = '../input/digit28-only-2-features-gkernel-bn-5-s102zz'
rs_init = 10201
for ii in range(30):
    rs = rs_init + ii
    y_pred_0, y_pred_test_0 = get_pred(src, rs)
    y_pred_list.append(y_pred_0)
    y_pred_test_list.append(y_pred_test_0)

In [ ]:
len(y_pred_list)

In [ ]:
n_slice = 70

y_pred = np.stack(y_pred_list)[:n_slice]
print(y_pred.shape)

pred = y_pred.mean(axis=0)
print(pred.shape)
print(pred[0])

In [ ]:
print(f1_score(y_train, np.argmax(pred, axis=1), average='macro'))
print(classification_report(y_train, np.argmax(pred, axis=1)))
confusion_matrix(y_train, np.argmax(pred, axis=1))

In [ ]:
y_pred_test = np.stack(y_pred_test_list)[:n_slice]
print(y_pred_test.shape)
pred_test = y_pred_test.mean(axis=0)
print(pred_test.shape)
print(pred_test[0])

submit_csv = sample_submit.copy()
submit_csv.Label = np.argmax(pred_test, axis=1)
submit_csv.head()
submit_csv.to_csv('submit_{}.csv'.format(n_slice), index=False)

In [ ]:
n_slice = 80

y_pred = np.stack(y_pred_list)[:n_slice]
print(y_pred.shape)

pred = y_pred.mean(axis=0)
print(pred.shape)
print(pred[0])

In [ ]:
print(f1_score(y_train, np.argmax(pred, axis=1), average='macro'))
print(classification_report(y_train, np.argmax(pred, axis=1)))
confusion_matrix(y_train, np.argmax(pred, axis=1))

In [ ]:
y_pred_test = np.stack(y_pred_test_list)[:n_slice]
print(y_pred_test.shape)
pred_test = y_pred_test.mean(axis=0)
print(pred_test.shape)
print(pred_test[0])

submit_csv = sample_submit.copy()
submit_csv.Label = np.argmax(pred_test, axis=1)
submit_csv.head()
submit_csv.to_csv('submit_{}.csv'.format(n_slice), index=False)

In [ ]:
y_pred = np.stack(y_pred_list)
print(y_pred.shape)

pred = y_pred.mean(axis=0)
print(pred.shape)
print(pred[0])

In [ ]:
print(f1_score(y_train, np.argmax(pred, axis=1), average='macro'))
print(classification_report(y_train, np.argmax(pred, axis=1)))
confusion_matrix(y_train, np.argmax(pred, axis=1))

In [ ]:
y_pred_test = np.stack(y_pred_test_list)
print(y_pred_test.shape)
pred_test = y_pred_test.mean(axis=0)
print(pred_test.shape)
print(pred_test[0])

submit_csv = sample_submit.copy()
submit_csv.Label = np.argmax(pred_test, axis=1)
submit_csv.head()
submit_csv.to_csv('submit.csv', index=False)